In [1]:
from deepface import DeepFace
import face_recognition
from PIL import Image
import os
from os import listdir
#from multiprocessing import Pool #mission impossible currently

In [2]:
import torch
import torchvision.transforms as transforms
import numpy as np
from numpy import asarray
import torch.nn.functional as F
from emotion_detection.fer_data_utils import SkResize, HistEq, AddChannel, ToRGB
from vision_utils.custom_architectures import SepConvModel
#extra added
from vision_utils.custom_architectures import SepConvModel, initialize_model, PretrainedMT
from multitask_rag.evaluate import predict_utk as test_image

In [3]:
PATHSEPCONV='/home/manish//Documents/democlassi/emotion-detection/sepconv_model_55_val_loss=1.175765.pth'
PATHRESNETAGR='/home/manish/Documents/democlassi/other/resnet_model_21_val_loss=4.275671.pth'

In [4]:
#emotion
model = SepConvModel()
model.load_state_dict(torch.load(PATHSEPCONV, map_location="cpu"))

#age-race-gender
resnet_model_agr = PretrainedMT(model_name='resnet')
resnet_model_agr.load_state_dict(torch.load(PATHRESNETAGR, map_location="cpu"))

<All keys matched successfully>

In [5]:
data_transforms = transforms.Compose([
    #transforms.Resize((500,500),Image.Resampling.LANCZOS),
    transforms.Resize((200,200),Image.Resampling.LANCZOS),
    transforms.ToTensor()
])

In [6]:
#these two function are needed doesn't work by just importing these function

In [7]:
def democlassiEvaluate(pil_image,):
    
    img_tensor= data_transforms(pil_image) 
    imge = pil_image.convert('L')
    img = imge.resize((120,120),Image.Resampling.LANCZOS)
    numpydata = asarray(img)
    res1=predict_fer(numpydata,model,True)
    # with resent model--age--race--gender
    #res2=test_image(img_tensor,resnet_model)
    #with sep_conv_model --age--gender---race
    #res2=test_image(img_tensor,sep_conv_model)
    #with VGG model ---age--race--gender
    #res2=test_image(img_tensor,vgg_model_agr)
    
    #for resnet model
    res2 = test_image(img_tensor,resnet_model_agr)
    
    #for sepconv_ model
    #res2 = test_image(img_tensor,sepconv_model_agr)
    return res1,res2 

In [8]:
def deepfaceOutput(obj1):
    print("Deepface comparision")
    print("----Deepface---")
    print("Age:---",{obj1['age']},"---")
    print("Gender:---",{obj1['gender']},"---")
    print("Race:---",{obj1['dominant_race']},"---")
    print("Emotion:---",{obj1['dominant_emotion']},"---")
    print('------------------------------------------- ')


In [9]:
def predict_fer(image, model, transf_learn=True):

    # process image
    image = preprocess_fer(image, transf_learn)
    
    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    emotion = F.softmax(model(image), dim=1).detach().to('cpu').numpy()[0]
    target_names = ['Angry', 'Disgusted', 'Afraid', 'Happy', 'Sad', 'Surprised', 'Neutral']
    pred_label = target_names[np.argmax(emotion)]

    emotion_probs = dict(zip(target_names, emotion))

    return emotion_probs, pred_label

In [10]:
def preprocess_fer(image, transf_learn):
    if transf_learn:
        transf = transforms.Compose([
            transforms.ToTensor()
        ])
    else:
        transf = transforms.Compose([
            HistEq(),
            AddChannel(),
            transforms.ToTensor()
        ])
        print(transf(image))  

    return transf(image).to(torch.float32).unsqueeze_(0)


In [11]:
#evalution stats

In [12]:
# create outer class
class Stats:
	def __init__(self,framework,gender):
		self.framework = framework
		self.gender = gender
		self.count = 0
		self.age = 0
        

	# create a 1st Inner class
	class Race:
		def __init__(self):
			self.white = 0
			self.black = 0
			self.asian = 0
			self.indian = 0
			self.unknown = 0


	# create a 2nd Inner class
	class Emotion:
		def __init__(self):
			self.angry = 0
			self.disgust = 0
			self.fear = 0
			self.happy = 0
			self.sad = 0
			self.suprise = 0
			self.neutral = 0

In [13]:
def printDemoClassiOutput(res1,res2):
    print("Democlassi comparision")
    print("---Democlassi---")
    print("Age:---",{round(res2[0], 0)},"---")
    print("Gender:---",{res2[2]},"---")
    print("Race:---",{res2[4]},"---")
    print("Emotion:---",{res1[1]},"---") #res1 for emotion

In [14]:
def deepfaceStats(stats,obj):
    stats.age = stats.age + obj['age']
    stats.count = stats.count + 1 
    
    if obj['dominant_race'] == "white":
        stats.race.white = stats.race.white+1
    elif obj['dominant_race']  == "black":
        stats.race.black = stats.race.black+1
    elif obj['dominant_race']  == "asian":
        stats.race.asian = stats.race.asian+1
    elif obj['dominant_race']  == "indian":
        stats.race.indian = stats.race.indian+1
    else:
        stats.race.unknown = stats.race.unknown+1
        
        
    if obj['dominant_emotion']== "angry":
        stats.emotion.angry= stats.emotion.angry+1
    elif obj ['dominant_emotion'] == "disgust":
        stats.emotion.disgust = stats.emotion.disgust+1
    elif obj['dominant_emotion']== "afraid":
        stats.emotion.afraid = stats.emotion.afraid+1
    elif obj['dominant_emotion']== "happy":
        stats.emotion.happy = stats.emotion.happy+1
    elif obj['dominant_emotion'] == "sad":
        stats.emotion.sad = stats.emotion.sad+1
    elif obj['dominant_emotion'] == "surprised":
        stats.emotion.suprise = stats.emotion.suprise+1
    else:
        stats.emotion.neutral = stats.emotion.neutral+1

In [15]:
def classiStats(stats,res1,res2):
    stats.age = stats.age + round(res2[0],0)
    stats.count = stats.count + 1 
    
    if res2[4] == "White":
        stats.race.white = stats.race.white+1
    elif res2[4] == "Black":
        stats.race.black = stats.race.black+1
    elif res2[4] == "Asian":
        stats.race.asian = stats.race.asian+1
    elif res2[4] == "Indian":
        stats.race.indian = stats.race.indian+1
    else:
        stats.race.unknown = stats.race.unknown+1
        
        
    if res1[1] == "Angry":
        stats.emotion.angry= stats.emotion.angry+1
    elif res1[1] == "Disgusted":
        stats.emotion.disgust = stats.emotion.disgust+1
    elif res1[1] == "Afraid":
        stats.emotion.afraid = stats.emotion.afraid+1
    elif res1[1] == "Happy":
        stats.emotion.happy = stats.emotion.happy+1
    elif res1[1] == "Sad":
        stats.emotion.sad = stats.emotion.sad+1
    elif res1[1] == "Surprised":
        stats.emotion.suprise = stats.emotion.suprise+1
    else:
        stats.emotion.neutral = stats.emotion.neutral+1

In [16]:
def evaluateDemoclassi(folder_dir, save_dir,statsmen,statswomen):
    for images in os.listdir(folder_dir):
        if (images.endswith(".png") or images.endswith(".jpg")or images.endswith(".jpeg")):
            new=folder_dir+"/"+images
            print(new)
            image = face_recognition.load_image_file(new)
            face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="resnet")
            if len(face_locations) != 0 :
                for face_location in face_locations:
                    top, right, bottom, left = face_location
                    face_image = image[top:bottom,left:right]
                    pil_img = Image.fromarray(face_image)
                    pil_image = pil_img.resize((400,400),Image.Resampling.LANCZOS)
                    res1, res2 = democlassiEvaluate(pil_image)
                    
                    if res2[2].lower() == "man" :
                        classiStats(statsmen,res1,res2)
                    else:
                        classiStats(statswomen,res1,res2)
                    #democlassiOutput(res1,res2)

In [22]:
def evaluateDeepface(folder_dir, save_dir,deepstatsMen,deepstatsWomen):
    for images in os.listdir(folder_dir):
        if (images.endswith(".png") or images.endswith(".jpg")or images.endswith(".jpeg")):
            new=folder_dir+"/"+images
            print(new)
            image = face_recognition.load_image_file(new)
            face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="resnet")
            if len(face_locations) != 0 :
                for face_location in face_locations:
                    top, right, bottom, left = face_location
                    face_image = image[top:bottom,left:right]
                    pil_img = Image.fromarray(face_image)
                    pil_image = pil_img.resize((400,400),Image.Resampling.LANCZOS)
                    fileLocation=save_dir+'/'+'tmp.jpg'
                    pil_image.save(fileLocation,"JPEG")
                    obj= DeepFace.analyze(img_path=fileLocation, enforce_detection=False)
                    
                    if obj["gender"] == "Man" :
                        deepfaceStats(deepstatsMen,obj)
                    else:
                        deepfaceStats(deepstatsWomen,obj)
                        
                    deepfaceOutput(obj)
                        
                    #printDemoClassiOutput(obj)
    #print('++++++++++++++++++++++++++++++++++++++++++')  

In [25]:
def printStats(stats):
    if stats.count  != 0 :
        print("Number of picture analyzed by the "+str(stats.count))
        averageAge = stats.age / stats.count 
        print("average age:" + str(averageAge))

        print("-----------------Race------------------")
        print("White: "+ str(stats.race.white))
        print("Black: " + str(stats.race.black))
        print("Asian:" +str(stats.race.asian))
        print("Indian: " + str(stats.race.indian))
        print("Unknown: " + str(stats.race.unknown))

        print("---------Emotion------------------")
        print("Angry: " + str(stats.emotion.angry))
        print("Disgust: " + str(stats.emotion.disgust))
        print("fear: " + str(stats.emotion.fear))
        print("Hppy: " + str(stats.emotion.happy))
        print("sad: " + str(stats.emotion.sad))
        print("Suprise:" + str(stats.emotion.suprise))
        print("neutral: " + str(stats.emotion.neutral) )
    else: 
        print("no stats for detected"+stats.framework +" for "+ stats.gender)




In [26]:
import time

folder_dir = "/home/manish/fotos-form-download" #folder containing picture location to be evaluate
save_dir= "/home/manish/Documents"   #saving of the temporary file
 

##democlassi statstics evaluation
demostatsMen = Stats("democlassi","men")
demostatsMen.race = Stats.Race()
demostatsMen.emotion = Stats.Emotion()

demostatsWomen= Stats("democlassi","women")
demostatsWomen.race = Stats.Race()
demostatsWomen.emotion = Stats.Emotion()

##deepface statstics evaluation
deepstatsMen = Stats("deepface","men")
deepstatsMen.race = Stats.Race()
deepstatsMen.emotion = Stats.Emotion()

deepstatsWomen= Stats("deepface","women")
deepstatsWomen.race = Stats.Race()
deepstatsWomen.emotion = Stats.Emotion()




#st = time.time()
#evaluateDemoclassi(folder_dir,save_dir,demostatsMen,demostatsWomen)
#et = time.time()


#printStats(demostatsMen)
#printStats(demostatsWomen)

#print("Time taken by democlassi  is : "+ str(et-st) + "seconds")


st = time.time()
evaluateDeepface(folder_dir,save_dir,deepstatsMen,deepstatsWomen)
et = time.time()

printStats(deepstatsMen)
printStats(deepstatsWomen)
print("Time taken by deepface  is : " +str(et-st)+ "seconds")



/home/manish/fotos-form-download/image327.jpg


Action: emotion:   0%|          | 0/4 [00:00<?, ?it/s]

1/1 [==============================] - 0s 24ms/step


Action: age:  25%|██▌       | 1/4 [00:00<00:00,  5.38it/s]    

1/1 [==============================] - 1s 1s/step


...

Action: race:  75%|███████▌  | 3/4 [00:01<00:00,  1.34it/s]  

1/1 [==============================] - 1s 756ms/step


Action: race: 100%|██████████| 4/4 [00:02<00:00,  1.41it/s]

Deepface comparision
----Deepface---
Age:--- {30} ---
Gender:--- {'Man'} ---
Race:--- {'white'} ---
Emotion:--- {'sad'} ---
------------------------------------------- 
/home/manish/fotos-form-download/image82.jpg


Number of picture analyzed by the 18
average age:30.666666666666668
-----------------Race------------------
White: 14
Black: 0
Asian:0
Indian: 1
Unknown: 3
---------Emotion------------------
Angry: 2
Disgust: 0
fear: 0
Hppy: 5
sad: 4
Suprise:0
neutral: 7
no object detected
Time taken by deepface  is : 64.23547625541687seconds
